In [117]:
import torch
import torch.nn as nn
import numpy as np
# from skimage import io, transform
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import os
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import unicodedata # ??
import nltk
from nltk.tokenize import TweetTokenizer
import csv
import json
from torchvision import transforms
from torch.autograd import Variable
np.random.seed(1)
random.seed(1)


# define directory structure needed for data processing
RAW_DATA_DIR = os.path.join('..', 'data/', 'raw_data/')
FORMAL_DATA_DIR = os.path.join('..', 'data/', 'formal_data/')
UNKNOWN_TOKEN = "unk"

## Split data into `train`, `val` and `test`
Split and write raw data as `acsii` format. 

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
def normalizeString(s):
    s = unicodeToAscii(s.strip()).replace("\t", "")
    return s


# precondition: two fields with name: "headline" and "text"
def splitData(fname, test_size=0.2, val_size=0.2): 
    df = pd.read_csv('../data/' + fname, encoding='latin-1')
    df = df[["headline", "text"]] # summary text, not the entire article
    df["headline"] = df["headline"].apply(normalizeString)
    df["text"] = df["text"].apply(normalizeString)

    df = df.sample(frac=1).reset_index(drop=True) # shuffle data

    df_train, df_test = train_test_split(df, test_size=test_size, random_state=1)
    df_train, df_val = train_test_split(df_train, test_size=val_size, random_state=1)
    df_train.to_csv(RAW_DATA_DIR + "train.csv", index=False, sep="\t")
    df_val.to_csv(RAW_DATA_DIR + "val.csv", index=False, sep="\t")
    df_test.to_csv(RAW_DATA_DIR + "test.csv", index=False, sep="\t")
    
# # TODO: check whether "text" is in fact the summary and corresponds to the headline

splitData("news_summary.csv")

## Preprocess `raw_data` to `formal_data`

In [ ]:
# class to store string transformation
class Transform(object):
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    @staticmethod
    def word_tokenize(s): 
        return json.dumps(Transform.tokenizer.tokenize(s))
    
    @staticmethod
    def cap(s): 
        return s.upper()
    
# tf: a transformation apply to each individual headline and text
# Apply a transformation to the dataframe in "raw_data" 
# Results of the transformation is in "formal_data"
def preprocess(fname, tf, chunksize=1000, num_chunk=-1): 
    header = True
    mode = "w"
    i = 0
    for df in pd.read_csv(RAW_DATA_DIR + fname, sep="\t", header = 0, chunksize=chunksize):
        df["headlines"] = df["headlines"].apply(tf)
        df["text"] = df["text"].apply(tf)
        df.to_csv(FORMAL_DATA_DIR+fname, columns = ['headlines','text'], mode=mode,index=False, header=header, sep="\t")
        if header == True:  
            # no header, and write in append mode from the 2nd chunk
            header = False; 
            mode = "a"
        
        i = i + 1 if i >= 0 else -1
        if i == num_chunk: 
            break; 
    

preprocess("train.csv", tf=Transform.word_tokenize)
preprocess("val.csv", tf=Transform.word_tokenize)
preprocess("test.csv", tf=Transform.word_tokenize)

## Language model from train data

In [ ]:
class GloVe():
    def __init__(self, path, dim):
        self.dim = dim
        self.word_embedding_dict = {}
        with open(path) as f:
            for line in f:
                values = line.split()
                embedding = values[-dim:]
                word = ''.join(values[:-dim])
                self.word_embedding_dict[word] = np.asarray(embedding, dtype=np.float32)
    
    def get_word_vector(self, word):
        if word not in self.word_embedding_dict.keys():
            embedding = np.random.uniform(low=-1, high=1, size=self.dim).astype(np.float32)
            self.word_embedding_dict[word] = embedding
            return embedding
        else:
            return self.word_embedding_dict[word]

In [ ]:
glvmodel = GloVe(os.path.join('..', 'models', 'glove.twitter.27B.200d.txt'), dim=200)

In [172]:
PAD_token = 0
SOS_token = 1 # start of sentence
EOS_token = 2 # end of sentence
PAD_word = "<PAD>"


# Modified from: Sean Robertson <https://github.com/spro/practical-pytorch>
class Lang: # language model
    def __init__(self, glvmodel, fname=None):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.gloveEmbed = []
        self.glvmodel = glvmodel
        self.size = 0  
        
        self.addWord(PAD_word)
        self.addWord("SOS")
        self.addWord("EOS")
        
        self.addWord(UNKNOWN_TOKEN)
        if fname is not None: 
            self.addCSV(fname)
        
    # add words of a sentence into the language model
    # split by ' '
    def addSentence(self, sentence):
        for word in sentence: 
            self.addWord(word)
            
    def addGlove(self, glove): 
        self.glove = glove

    def getGloveLayer(self, embed_dim): 
        # Initialize word embeddings from our pre-training GloVe embeddings
        glove_embeddings = torch.from_numpy(self.gloveEmbed)
        return nn.Embedding(self.size, embed_dim).from_pretrained(glove_embeddings, freeze=False)
        
    # add a word to language model
    def addWord(self, word):
        if word not in self.word2index:
            # assign id for word
            self.word2index[word] = self.size
            
            # count word
            self.word2count[word] = 1
            
            self.index2word[self.size] = word
            self.size += 1
            
#             gloveEmbed.append(glvmodel.get_word_vector(word))
        else:
            self.word2count[word] += 1
            
    def addDataFrame(self, df): 
        # df: headline, text
        for index, row in df.iterrows():
            self.addSentence(json.loads(row['headlines']))
            self.addSentence(json.loads(row['text']))
            
    def addCSV(self, fname): 
        # construct language model based on a file
        # file is a dataframe csv file with "headlines" and "text"
        for df in pd.read_csv(FORMAL_DATA_DIR + fname, sep="\t", header = 0, chunksize=2000):
            self.addDataFrame(df)     
    
    def size(self): 
        return len(self.word2index)
    
    def wordSeq2IdxSeq(self, word_seqs):
        default_idx = lang.word2index[UNKNOWN_TOKEN]
        idxs = []
        for word_seq in word_seqs:
            idxs.append([lang.word2index.get(w, default_idx) for w in word_seq])
    #         idxs.append(torch.LongTensor([lang.word2index.get(w, default_idx) for w in word_seq]))
        return idxs

lang = Lang([])
lang.addCSV("train.csv")

## Neural Network model

In [118]:
def init_param(param, bias_std=0.0, weight_std=0.05): 
    for name, param in self.gru.named_parameters():
        if 'bias' in name:
            nn.init.constant_(param, bias_std)
        elif 'weight' in name:
            nn.init.normal_(param, std = weight_std)
            
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=self.dropout, bidirectional=True)
        
    def forward(self, input_seqs, input_lengths, hidden=None):
        # Note: we run this all at once (over multiple batches of multiple sequences)
        embedded = self.embedding(input_seqs)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
        return outputs, hidden
    
class Encoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, 
                    pretrained_embeddings, batch_size, num_layers):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.word_embeddings = gloveLayer
        
        # Initialize a Gated Recurrent Unit RNN
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers)
        
#         self.hidden2label = nn.Linear(hidden_dim, n_classes)
        self.hidden = self.init_hidden()
        
        # Custom initialization of the weights and biases
        init_param(self.gru.named_parameters())
#         init_param(self.hidden2label.named_parameters())


    def init_hidden(self):
        return Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).cuda())

    
#     def forward(self, input_seqs):
        
#         embedded = self.word_embeddings(input_seqs)
#         packed_embeds = nn.utils.rnn.PackedSequence(input_embed, packed_sequence.batch_sizes)
#         output, hidden = self.gru(packed_embeds, self.hidden)
#         return output, hidden
    
    def forward(self, input_seqs, input_lengths, hidden=None):
        # Note: we run this all at once (over multiple batches of multiple sequences)
        embedded = self.word_embeddings(input_seqs)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
        return outputs, hidden

class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        
        self.gru = nn.GRU(hidden_size, hidden_size)
        
        # map to output space
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1) # ? what is LogSoftMax, dim = 1??

    def forward(self, input, hidden):
        # ? why the input is is 'output_size' dimension? 
        # ? why need embedding? 
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### Training
#### Dataset

In [3]:
class SummaryDataset(Dataset):
    def __init__(self, fname, transform=None):
        self.df = pd.read_csv(FORMAL_DATA_DIR + fname, sep="\t", header = 0)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        headline = self.df["headlines"][idx]
        text = self.df["text"][idx]
        
        sample = {'headlines': headline, 'text': text}

        if self.transform:
            sample = self.transform(sample)

        return sample

# trainDataset = SummaryDataset(FORMAL_DATA_DIR + "train.csv")

In [192]:
# class ToTensor(object):
#     """Convert the pair of string lists to Tensors."""
#     def __init__(self, lang): 
#         self.lang = lang
        
#     def __call__(self, sample):
#         headlines, text = sample['headlines'], sample['text']
#         headlines = prepare_sequence_batch(headlines, self.lang)
#         text = prepare_sequence_batch(text, self.lang)
#         return (torch.Tensor(headlines), torch.Tensor(text))


# Pad a with the PAD symbol
def pad_seq(seq, max_length):
    seq += [PAD_token for i in range(max_length - len(seq))]
    return seq


"""
Create a batch ready for feeding into the network. 
Paddings are added to ensure all sequences are the same length. 
Precondition: 
    input_seqs: a list of sequences; each element is a sequencce which is a list of words
    target_seqs: same. 
    lang: the language model
Postcondition: 
    indices version of the input and target in tensor form. 
"""
def batch(input_seqs, target_seqs, lang):
    # input_seqs and target_seqs are in string format
    
    input_seqs = lang.wordSeq2IdxSeq(input_seqs)
    target_seqs = lang.wordSeq2IdxSeq(target_seqs)
    
    print(target_seqs)
    
    # Zip into pairs, sort by length (descending), unzip
    seq_pairs = sorted(zip(input_seqs, target_seqs), key=lambda p: len(p[0]), reverse=True)
    input_seqs, target_seqs = zip(*seq_pairs)
    
    # For input and target sequences, get array of lengths and pad with 0s to max length
    input_lengths = [len(s) for s in input_seqs]
    input_padded = [pad_seq(s, max(input_lengths)) for s in input_seqs]
    target_lengths = [len(s) for s in target_seqs]
    target_padded = [pad_seq(s, max(target_lengths)) for s in target_seqs]

    for s in input_padded: 
        print(s)
        
    for s in target_padded: 
        print(s)
    # Turn padded arrays into (batch_size x max_len) tensors, transpose into (max_len x batch_size)
    input_var = Variable(torch.LongTensor(input_padded)).transpose(0, 1)
    target_var = Variable(torch.LongTensor(target_padded)).transpose(0, 1)
    
#     input_var = input_var.cuda()
#     target_var = target_var.cuda()
    
    return input_var, input_lengths, target_var, target_lengths



# tweet_in = nn.utils.rnn.pack_sequence(prepare_sequence_batch(headlines, word2idx)).cuda()
# target = nn.utils.rnn.pack_sequence(prepare_sequence_batch(text, word2idx)).cuda()

In [197]:
# tf = transforms.Compose([ToTensor(lang)])
tf = None
dataset = SummaryDataset(fname="train.csv", transform=tf)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)


sb = next(iter(dataloader))
input_seqs = sb["text"]
target_seqs = sb["headlines"]
input_seqs = [json.loads(s) for s in input_seqs]
target_seqs = [json.loads(s) for s in target_seqs]
input_seqs, input_lengths, target_seqs, target_lengths = batch(input_seqs, target_seqs, lang)


for i_batch, sample_batched in enumerate(dataloader):
    # clear gradients, clear hidden state from the last timestep
#     gru_model.zero_grad()
#     gru_model.hidden = gru_model.init_hidden()

    input_seqs = sample_batched["text"]
    target_seqs = sample_batched["headlines"]
    
    input_seqs = [json.loads(s) for s in input_seqs]
    target_seqs = [json.loads(s) for s in target_seqs]
    
    input_seqs, input_lengths, target_seqs, target_lengths = batch(input_seqs, target_seqs, lang)
    
    input_seqs = nn.utils.rnn.pack_sequence(input_seqs) # .cuda()
    target_seqs = nn.utils.rnn.pack_sequence(target_seqs) #.cuda()

#     # forward pass
#     label_scores = gru_model(tweet_in)

#     # compute loss against true labels
#     loss = loss_function(label_scores, target)

#     # backprop the gradients and update the model parameters
#     loss.backward()
#     optimizer.step()

#     # keep track of the loss
#     running_loss += loss.item()
#     i += BATCH_SIZE
#     if i % 2000 == 0:
#         average_loss = running_loss/2000
#         if average_loss < lowest_loss:
#             lowest_loss = running_loss
#             # save our checkpoint if it is the current best
#             torch.save(gru_model.state_dict(), CHECKPOINT_FILE)
#         logging.info("running loss: %.3f @ batch %d", average_loss, batch_ind)
#         running_loss = 0.0

[[324, 1851, 863, 6360, 769, 6361, 191, 318, 1906, 59, 2068], [12754, 11440, 11182, 12755, 246, 899, 3724, 203, 120, 1727, 3520, 120], [4501, 2378, 59, 155, 2414, 59, 3712, 7316, 7, 2167, 1052], [5174, 11, 259, 4929, 17436, 10609, 17437]]
[3897, 12754, 11440, 11481, 217, 11441, 11442, 11, 90, 4873, 16, 387, 12755, 246, 899, 7, 30, 3897, 50, 81, 195, 120, 1727, 3520, 120, 37, 97, 11, 81, 4595, 59, 92, 12756, 132, 12757, 478, 30, 3897, 235, 47, 2305, 1249, 37, 11441, 11442, 50, 47, 83, 1421, 18, 30, 11481, 195, 1304, 29, 2157, 246, 12758, 59, 20, 30, 12759, 132, 8679, 7, 482, 5063, 20, 37]
[2101, 47, 105, 75, 1384, 274, 2378, 59, 3712, 7316, 7, 2167, 1052, 11, 651, 13282, 2668, 143, 3106, 14012, 990, 132, 120, 1203, 196, 9647, 120, 7316, 1009, 29, 7236, 7686, 143, 1270, 37, 83, 1676, 2704, 3021, 1471, 10083, 253, 11, 234, 75, 1644, 2378, 59, 3912, 218, 7, 6245, 1627, 11, 75, 92, 2317, 37, 30, 2767, 16, 387, 6393, 89, 30, 105, 558, 37, 0]
[1454, 4791, 651, 5174, 5175, 132, 261, 262, 263, 

[[3845, 3846, 3847, 3848, 191, 381, 692, 384, 986, 707, 294], [5860, 584, 5858, 196, 59, 60, 782, 44, 3474, 1460], [2714, 4560, 47, 287, 82, 83, 1968, 1076, 1789, 2037, 2175, 3678], [172, 2165, 143, 6203, 11388, 5994, 10698, 1811, 47, 969]]
[386, 382, 2187, 383, 384, 11, 153, 195, 294, 89, 388, 389, 11, 50, 20, 3849, 3850, 3851, 11, 3849, 525, 11, 3845, 3846, 3847, 3848, 11, 3852, 3853, 3854, 39, 1975, 38, 985, 253, 37, 3855, 41, 20, 246, 707, 985, 37, 19, 175, 18, 19, 1657, 3856, 11, 132, 211, 235, 47, 117, 3857, 75, 3858, 59, 28, 37, 20, 3850, 3859, 78, 3860, 3861, 39, 99, 1009, 782, 143, 3862, 41, 11, 20, 19, 175, 37]
[1906, 160, 5859, 5860, 16, 1316, 5858, 11, 30, 17145, 1147, 153, 195, 1586, 67, 819, 59, 252, 11, 196, 59, 60, 482, 328, 44, 3474, 1460, 37, 270, 604, 246, 481, 1201, 1989, 29, 68, 553, 143, 83, 5767, 1647, 67, 17146, 3971, 143, 5708, 132, 2081, 9697, 11, 399, 234, 83, 13187, 1573, 195, 1486, 132, 481, 195, 886, 217, 83, 3013, 37, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

[[15731, 12, 2716, 601, 482, 120, 15732, 120, 375, 3331], [441, 1399, 747, 4387, 11, 3014, 11418, 986, 14380], [498, 1015, 82, 1016, 47, 392, 132, 382, 191, 500], [3760, 3593, 83, 733, 59, 14597, 715, 191, 160]]
[2342, 193, 15731, 7216, 16, 17, 47, 83, 4612, 18, 2716, 601, 83, 120, 15732, 120, 561, 7, 15733, 67, 375, 37, 47, 30, 561, 11, 15731, 957, 92, 912, 4218, 82, 83, 2895, 59, 3841, 30, 31, 120, 2895, 752, 363, 2896, 120, 37, 481, 550, 11, 20, 3108, 675, 2227, 303, 3108, 3251, 47, 37, 3108, 599, 1482, 2227, 18, 303, 3270, 675, 808, 3268, 14815, 2228, 20, 213]
[30, 500, 29, 1017, 50, 18, 1018, 42, 498, 1015, 82, 243, 1019, 1016, 47, 392, 132, 382, 132, 30, 1020, 1021, 558, 235, 20, 491, 495, 20, 37, 20, 30, 23, 160, 235, 1022, 47, 1023, 541, 808, 1024, 42, 1025, 132, 30, 1026, 726, 37, 1018, 235, 498, 1027, 11, 20, 50, 500, 37, 30, 1016, 99, 97, 592, 500, 7, 1028, 81, 47, 30, 429, 458, 37, 0]
[83, 18211, 2787, 635, 11, 153, 195, 747, 4387, 89, 83, 7885, 18212, 217, 83, 3058, 47, 32

[[1658, 10897, 140, 30, 3038, 5514, 191, 2593, 2594, 59, 9720, 2566], [7735, 3774, 7739, 191, 6654, 7733, 2213, 1703, 287], [82, 17799, 31, 4238, 11, 698, 17800, 589, 143, 106, 1318], [8022, 5235, 2461, 1109, 5291, 217, 253, 2037, 916, 191, 724]]
[287, 726, 727, 724, 16, 421, 18, 5763, 5235, 6809, 8023, 20, 2461, 1109, 5291, 217, 253, 20, 1023, 37, 277, 11, 724, 50, 20, 30, 3404, 20, 16, 8024, 351, 7, 30, 287, 531, 8025, 532, 37, 117, 2432, 1686, 47, 30, 5183, 7, 8023, 1592, 2356, 7, 144, 5770, 771, 67, 30, 525, 234, 195, 20, 8026, 20, 29, 5647, 132, 8027, 7, 10, 4162, 59, 30, 456, 37]
[82, 17799, 31, 4238, 47, 1614, 17801, 8509, 11, 698, 16, 17802, 30, 589, 143, 30, 106, 1318, 57, 30, 1801, 1063, 7, 4238, 47, 12111, 6774, 217, 17803, 37, 30, 107, 1896, 7, 428, 11, 7404, 11, 7062, 11, 31, 132, 17804, 31, 1438, 29, 103, 1500, 3205, 270, 3198, 37, 47, 4475, 10, 4822, 11, 698, 2363, 8970, 4621, 246, 1945, 757, 1454, 4811, 395, 37, 0, 0, 0]
[2538, 351, 7, 117, 1390, 4539, 29, 30, 5902, 217

[[3217, 2539, 1329, 7, 9254, 12986, 12984, 17207], [105, 1099, 3468, 1482, 274, 3630, 246, 1114, 726], [287, 996, 59, 518, 424, 120, 7216, 7166, 120, 217, 9034], [6726, 8152, 4044, 143, 144, 9063, 647, 59, 4738, 144, 4839, 771]]
[83, 6726, 8152, 235, 647, 59, 4738, 4176, 14467, 39, 144, 4839, 771, 41, 217, 35, 3999, 47, 2710, 11, 995, 554, 246, 243, 3956, 4044, 81, 143, 4176, 145, 39, 144, 9063, 41, 11, 14468, 81, 195, 83, 960, 14469, 37, 30, 7365, 4107, 30, 14470, 14471, 6726, 143, 9446, 11, 1217, 4701, 243, 3111, 246, 83, 14472, 50, 81, 459, 92, 14473, 37, 30, 7365, 42, 20, 14474, 12307, 11, 20, 83, 14472, 50, 37]
[30, 287, 558, 16, 1106, 59, 518, 424, 83, 120, 7216, 7166, 120, 217, 9035, 9036, 1208, 59, 3841, 3973, 541, 47, 30, 7216, 8234, 11, 287, 3788, 132, 11130, 160, 1440, 1432, 50, 37, 30, 107, 558, 75, 1393, 327, 144, 1539, 146, 59, 30, 2118, 4986, 11, 19, 175, 37, 30, 7166, 75, 6836, 752, 30, 2679, 7, 8234, 11, 4011, 132, 3788, 4622, 7, 9034, 37, 0, 0, 0, 0, 0, 0, 0, 0]
[1720

...

[[2068, 2018, 70, 59, 2140, 8438, 2494, 7675], [10541, 10542, 4839, 59, 92, 363, 245, 10541, 10542, 31, 191, 10543], [1772, 1250, 17769, 59, 1658, 3112, 67, 8660, 10123, 4744], [810, 287, 1699, 12257, 59, 1672, 12258, 3024]]
[2264, 10544, 10543, 16, 50, 18, 120, 10541, 10542, 4839, 120, 75, 92, 117, 245, 31, 47, 30, 120, 10541, 10542, 120, 31, 809, 37, 19, 175, 18, 246, 270, 31, 11, 3042, 4779, 459, 208, 30, 7103, 3606, 206, 81, 16, 59, 92, 10545, 1179, 37, 30, 31, 75, 1462, 30, 6948, 7, 5600, 6375, 57, 120, 10541, 10542, 120, 132, 10546, 10547, 57, 117, 10548, 120, 9665, 120, 37]
[30, 329, 1276, 29, 509, 1142, 70, 59, 2126, 243, 2222, 7, 344, 29, 30, 8439, 8438, 2494, 7675, 37, 20, 1775, 2631, 720, 600, 92, 3205, 957, 92, 1106, 1206, 11, 20, 30, 1276, 175, 132, 2589, 30, 183, 2081, 143, 1004, 4925, 37, 466, 11, 5248, 5450, 243, 8440, 7, 8441, 30, 7675, 11, 234, 75, 4329, 243, 2222, 7, 8442, 720, 2256, 70, 37, 0, 0, 0, 0]
[5029, 112, 16865, 1895, 16, 2543, 83, 15611, 111, 18, 1250, 30,

[[818, 806, 235, 30, 281, 7492, 47, 125, 1182, 191, 1168], [6164, 15085, 11, 15086, 175, 59, 15087, 5546], [3888, 631, 1217, 26, 457, 235, 47, 1369, 191, 160], [2258, 2218, 12, 35, 6338, 3439, 836, 195, 1060, 28]]
[2563, 8221, 160, 15189, 16819, 29, 1017, 50, 18, 3888, 2685, 10200, 1217, 26, 30, 457, 235, 47, 1369, 37, 20, 3270, 30, 1369, 235, 410, 11, 42, 686, 433, 1789, 144, 20, 19, 52, 37, 20, 15765, 3888, 460, 82, 16820, 37, 498, 4319, 16, 2654, 2486, 362, 47, 3335, 1338, 168, 30, 16821, 47, 16822, 140, 30, 5329, 16823, 47, 8691, 11, 20, 19, 175, 37]
[410, 5096, 90, 4339, 11, 651, 6164, 15085, 132, 15086, 11, 293, 709, 175, 59, 30, 15087, 5546, 37, 15088, 11, 9732, 59, 30, 2972, 7, 5039, 15089, 11, 132, 3440, 2510, 15090, 293, 176, 175, 37, 7083, 11, 30, 2392, 15091, 195, 462, 83, 90, 5545, 342, 59, 15092, 1349, 1482, 63, 2507, 82, 3042, 89, 498, 499, 3015, 140, 2724, 59, 2304, 132, 884, 37, 0]
[1172, 1174, 1168, 16, 17, 18, 6479, 818, 806, 132, 4441, 4435, 42, 30, 281, 1525, 132, 

[[3675, 3676, 3677, 11, 1467, 3217, 748, 294], [21, 22, 12321, 12322, 31, 410, 2392, 3231, 191, 836], [644, 1123, 1864, 2996, 2388, 57, 17168, 132, 4740], [500, 402, 12028, 1482, 47, 2618, 2646, 726, 2649, 544, 144, 1549, 12029]]
[57, 305, 306, 11, 30, 21, 22, 16, 12323, 30, 2127, 7, 30, 12324, 30, 12325, 125, 11, 234, 235, 29, 4007, 12326, 12322, 12327, 11, 410, 30, 997, 7, 4339, 120, 3231, 120, 11, 120, 2298, 120, 11, 120, 12328, 9026, 7, 252, 120, 132, 120, 7763, 252, 120, 37, 30, 12329, 583, 12330, 12331, 50, 18, 19, 339, 3108, 3939, 59, 12332, 140, 5863, 2044, 18, 12322, 16, 50, 47, 30, 12324, 37]
[500, 402, 293, 985, 12030, 1482, 34, 773, 674, 1026, 726, 2649, 418, 2655, 12031, 12032, 12033, 47, 3128, 29, 509, 234, 195, 521, 59, 2108, 4772, 2109, 37, 2618, 160, 12034, 12035, 132, 482, 699, 636, 353, 418, 745, 4478, 11370, 34, 30, 995, 342, 1736, 428, 746, 37, 12034, 99, 798, 682, 39, 683, 41, 132, 99, 12036, 472, 674, 457, 725, 29, 500, 3737, 37, 0, 0, 0, 0, 0, 0, 0]
[30, 5103, 5

[[369, 434, 2771, 2030, 3938, 4662, 191, 5891], [819, 3332, 5671, 3239, 5672, 246, 799, 5673, 447], [17706, 17707, 956, 1358, 3126, 246, 17708, 10036], [9105, 2788, 11, 5117, 6412, 7255, 246, 10440, 47, 589]]
[125, 5674, 3239, 5672, 11, 1502, 2325, 2334, 209, 709, 11, 5675, 5676, 246, 83, 1130, 618, 3332, 5677, 125, 3937, 399, 2309, 906, 2359, 189, 249, 252, 29, 1017, 37, 30, 1130, 3332, 195, 5598, 1370, 11, 20, 5678, 5679, 5680, 144, 20, 11, 234, 235, 26, 5672, 370, 117, 5681, 132, 4841, 1304, 860, 30, 1130, 3332, 132, 5682, 217, 28, 986, 802, 794, 521, 28, 37]
[5892, 29, 1356, 5893, 618, 1354, 3508, 209, 1563, 11, 773, 801, 5894, 5891, 50, 18, 5895, 1203, 196, 2771, 2030, 3938, 57, 2631, 57, 78, 837, 47, 30, 3537, 37, 20, 369, 600, 92, 4847, 59, 3572, 752, 45, 5896, 11, 196, 468, 217, 782, 11, 20, 19, 175, 37, 466, 11, 5895, 293, 5897, 5898, 89, 810, 811, 47, 30, 709, 2568, 807, 809, 37, 0, 0]
[30, 956, 2998, 12111, 1358, 7, 5335, 5336, 17706, 784, 11, 234, 195, 2636, 59, 4203, 183, 

[[9052, 996, 2210, 438, 719, 191, 2068, 29, 525, 996, 144, 1549, 2433], [1124, 9385, 3021, 443, 3368, 82, 10938, 120, 10939, 10940, 120], [2362, 2679, 443, 120, 2987, 9925, 120, 143, 267, 10008], [4782, 59, 2100, 15465, 120, 2691, 15466, 82, 9295, 14482, 66]]
[10941, 11, 35, 10942, 9385, 1505, 47, 819, 11, 29, 824, 448, 152, 83, 10034, 59, 2222, 274, 1749, 82, 83, 20, 10939, 10940, 20, 140, 138, 10938, 37, 10941, 50, 11, 20, 202, 957, 5184, 30, 10940, 274, 10943, 132, 30, 10940, 75, 1206, 155, 47, 8553, 82, 2154, 82, 83, 6505, 10939, 37, 20, 81, 97, 1486, 20, 10941, 3876, 11, 20, 9681, 59, 1820, 3145, 10944, 59, 3112, 120, 10945, 37]
[30, 15467, 4782, 558, 235, 2958, 59, 1683, 164, 14482, 66, 59, 15468, 15465, 11, 8664, 274, 9910, 59, 13662, 9275, 89, 1339, 37, 30, 768, 235, 20, 59, 127, 4850, 15465, 1864, 5803, 11, 140, 1467, 15469, 67, 3884, 274, 3459, 7512, 2316, 8404, 15470, 11, 20, 978, 50, 37, 270, 604, 1306, 83, 809, 7, 12481, 15471, 67, 2167, 132, 5952, 59, 970, 663, 15465, 37,

In [202]:
print(target_seqs)

PackedSequence(data=tensor([  672,   324, 12876,    47,  2561,   168,    83, 12877,  9996,   191,
        11965,  1350,  7516,   584,  2904,   402,    59,   717,   249,   445,
           47,   470,   471,     0,   410,  1050,   287,  1648,  1789,  1989,
          993,  1851,    47,  3723,     0,     0,   897,  1056,  4901, 12218,
           47, 12219,   250,  4635,     0,     0,     0,     0]), batch_sizes=tensor([12, 12, 12, 12]))


In [194]:
# tf = transforms.Compose([ToTensor(lang)])
tf = None
dataset = SummaryDataset(fname="train.csv", transform=tf)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)


sb = next(iter(dataloader))
input_seqs = sb["text"]
target_seqs = sb["headlines"]
input_seqs = [json.loads(s) for s in input_seqs]
target_seqs = [json.loads(s) for s in target_seqs]
input_seqs, input_lengths, target_seqs, target_lengths = batch(input_seqs, target_seqs, lang)

[[1556, 196, 4575, 9260, 9261, 11, 3262, 363, 14839, 5288, 191, 14840], [14566, 132, 4761, 7, 1897, 82, 252, 11, 1124, 1337, 191, 589], [473, 3724, 381, 144, 1549, 2989, 217, 183, 3055, 47, 2305, 1249, 191, 259, 29, 16014], [1412, 2222, 953, 7, 120, 4077, 8729, 120, 29, 375]]
[261, 262, 263, 11, 757, 2321, 1060, 10521, 10522, 10523, 16015, 39, 16014, 41, 1390, 5180, 11, 50, 11, 20, 47, 2305, 1249, 11, 303, 339, 1658, 30, 1384, 11, 140, 3724, 7154, 2989, 217, 30, 183, 3055, 37, 20, 30, 5180, 1429, 6260, 1730, 860, 259, 11, 153, 235, 29, 83, 1394, 1384, 37, 19, 175, 11, 20, 303, 339, 3108, 1203, 81, 47, 2305, 1249, 11, 3647, 363, 1287, 6, 11, 5880, 482, 16016, 37, 20]
[260, 14840, 14841, 16, 1610, 5805, 7, 4575, 9260, 9261, 132, 50, 18, 3262, 117, 14839, 5288, 37, 19, 175, 11, 20, 369, 42, 803, 1750, 47, 1464, 8982, 2445, 37, 481, 16, 35, 6931, 9617, 7, 14842, 11, 132, 303, 276, 2590, 1318, 82, 482, 37, 20, 5805, 7, 30, 311, 4575, 660, 26, 78, 345, 1201, 6, 143, 5622, 29, 14843, 1233, 47

In [ ]:

def train(input_batches, input_lengths, target_batches, target_lengths, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    
    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 # Added onto for each word

    # Run words through encoder
    encoder_outputs, encoder_hidden = encoder(input_batches, input_lengths, None)
    
    # Prepare input and output variables
    decoder_input = Variable(torch.LongTensor([SOS_token] * batch_size))
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder

    max_target_length = max(target_lengths)
    all_decoder_outputs = Variable(torch.zeros(max_target_length, batch_size, decoder.output_size))

    # Move new Variables to CUDA
    if USE_CUDA:
        decoder_input = decoder_input.cuda()
        all_decoder_outputs = all_decoder_outputs.cuda()

    # Run through decoder one time step at a time
    for t in range(max_target_length):
        decoder_output, decoder_hidden, decoder_attn = decoder(
            decoder_input, decoder_hidden, encoder_outputs
        )

        all_decoder_outputs[t] = decoder_output
        decoder_input = target_batches[t] # Next input is current target

    # Loss calculation and backpropagation
    loss = masked_cross_entropy(
        all_decoder_outputs.transpose(0, 1).contiguous(), # -> batch x seq
        target_batches.transpose(0, 1).contiguous(), # -> batch x seq
        target_lengths
    )
    loss.backward()
    
    # Clip gradient norms
    ec = torch.nn.utils.clip_grad_norm(encoder.parameters(), clip)
    dc = torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)

    # Update parameters with optimizers
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0], ec, dc

In [ ]:
DF = []
for df in pd.read_csv(RAW_DATA_DIR + 'train.csv', sep="\t", header = 0, chunksize=10):
    DF  = df
    break